Text Generator

In [0]:
import numpy
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils


Using TensorFlow backend.


In [0]:
#Code to read file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

link = 'https://drive.google.com/open?id=1w4lOacBmOtly-ozf8Ncfrl4oCL4minZ8'

fluff, id = link.split('=')
print (id) # Verify that you have everything after '='
downloaded = drive.CreateFile({'id':id}) 
raw_text = downloaded.GetContentString()



1w4lOacBmOtly-ozf8Ncfrl4oCL4minZ8


In [0]:

raw_text = raw_text.lower()

In [0]:
#print(raw_text)

In [0]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [0]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print ("Total Characters: ", n_chars)
print ("Total Vocab: ", n_vocab)

Total Characters:  94275
Total Vocab:  38


In [0]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = raw_text[i:i + seq_length]
	seq_out = raw_text[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)


Total Patterns:  94175


In [0]:
def perplexity(y_true, y_pred):
    
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    perplexity = K.exp(cross_entropy)
    return perplexity


def crossentropy(y_true, y_pred):
    return K.categorical_crossentropy(y_true, y_pred)

In [0]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable

y = np_utils.to_categorical(dataY)
print(X.shape)
print(y.shape)

(94175, 100, 1)
(94175, 38)


In [0]:
# define the LSTM model
model = Sequential()
model.add(LSTM(512, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(512))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[crossentropy, perplexity])





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [0]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 38)                19494     
Total params: 3,171,366
Trainable params: 3,171,366
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
# fit the model
model.fit(X, y, epochs=50, batch_size=2000)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/50





94175/94175 [==============================] - 70s 740us/step - loss: 3.0983 - crossentropy: 3.0983 - perplexity: 34.5722
Epoch 2/50
94175/94175 [==============================] - 62s 662us/step - loss: 2.9949 - crossentropy: 2.9949 - perplexity: 35.6375
Epoch 3/50
94175/94175 [==============================] - 62s 662us/step - loss: 2.9872 - crossentropy: 2.9872 - perplexity: 35.7610
Epoch 4/50
94175/94175 [==============================] - 62s 663us/step - loss: 2.9550 - crossentropy: 2.9550 - perplexity: 36.3816
Epoch 5/50
94175/94175 [==============================] - 62s 663us/step - loss: 2.8611 - crossentropy: 2.8611 - perplexity: 36.3038
Epoch 6/50
94175/94175 [==============================] - 62s 662us/step - loss: 2.7927 - crossentropy: 2.7927 - perplexity: 36.1388
Epoch 7/50
94175/94175 [==============================] - 62s 662us/step - loss: 2.7378 - crossentro

In [0]:
# def sample(preds, temperature=1.0):
#     # helper function to sample an index from a probability array
#     preds = np.asarray(preds).astype('float64')
#     preds = np.log(preds) / temperature
#     exp_preds = np.exp(preds)
#     preds = exp_preds / np.sum(exp_preds)
#     probas = np.random.multinomial(1, preds, 1)
#     return np.argmax(probas)

In [0]:
import sys
import numpy as np

# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]


print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
	x = np.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = np.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	sys.stdout.write(result)
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print ("\nDone.")

Seed:
"  state,
and trouble deaf heaven with my bootless cries,
and look upon my self and curse my fate,
wis "
hin the love thee sit she surength of lays,
so should the steasure of thy silf desengs,
and that in thee that shou dost coom the shee,
which sake to booptait of the ruimed shile,
then that meke bea for me wou in my heart,
and meke to thee to fave then my sooe spuessed,
but thos art to see shale of your seroest,
and that in thee this ioows that thou drst st,
to fiat to make thes born that i worlds,
which in the world s secked shake i am standed

to me that live in thee i am forg toon.



that thie of this allne tit doth reme so thee.



mn the danl of this blund with this gave thee mort in maru so the wirl and sase  

but thou the world will be thy love and stand,
but thou the sime thes seal the strlng ouce,
and to the part of lapu siat in my beees,
and lekd my self to beauty sersed mone dace,
and therefore layst are soeek of so the seare,
and beauty shall beauty leke to me ae 